In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

# Arguments

In [2]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [3]:
pretrained = ''
resume = ''

In [4]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 400
start_epoch = 0
train_batch = 256
test_batch = 200
lr = 0.04
schedule = [150, 275]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [14]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [16]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in schedule:
        state['lr'] *= gamma
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    adjust_learning_rate(optimizer, epoch)
    lr = optimizer.state_dict()['param_groups'][0]['lr']
    state['lr'] = lr
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 400] LR: 0.040000
1/269 Data:3.659 | Batch:8.118 | Total:0:00:08 | ETA:0:36:16 | Loss:0.6927245259284973 | top1:53.515625
11/269 Data:0.003 | Batch:0.505 | Total:0:00:13 | ETA:0:05:47 | Loss:0.7046850594607267 | top1:50.7457389831543
21/269 Data:0.003 | Batch:0.566 | Total:0:00:19 | ETA:0:02:22 | Loss:0.8049656095958891 | top1:50.42782974243164
31/269 Data:0.004 | Batch:0.662 | Total:0:00:28 | ETA:0:03:30 | Loss:0.7856161594390869 | top1:50.02519989013672
41/269 Data:0.002 | Batch:0.658 | Total:0:00:36 | ETA:0:03:02 | Loss:0.765358823101695 | top1:50.11432647705078
51/269 Data:0.001 | Batch:0.567 | Total:0:00:43 | ETA:0:02:24 | Loss:0.7515898942947388 | top1:50.32935333251953
61/269 Data:0.079 | Batch:0.642 | Total:0:00:50 | ETA:0:02:44 | Loss:0.7471849937907985 | top1:50.384220123291016
71/269 Data:0.006 | Batch:0.689 | Total:0:01:02 | ETA:0:03:44 | Loss:0.7412598401727811 | top1:50.39612579345703
81/269 Data:0.001 | Batch:0.552 | Total:0:01:11 | ETA:0:02:47 | Loss:0.7377

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


39/39 Data:0.000 | Batch:0.104 | Total:0:00:29 | ETA:0:00:00 | Loss:0.6935396026342343 | top1:50.0

Epoch: [2 | 400] LR: 0.068000
1/269 Data:4.757 | Batch:5.510 | Total:0:00:05 | ETA:0:24:37 | Loss:0.6923876404762268 | top1:50.78125
11/269 Data:0.001 | Batch:0.562 | Total:0:00:14 | ETA:0:06:00 | Loss:0.7872607491233132 | top1:49.5738639831543
21/269 Data:0.009 | Batch:0.530 | Total:0:00:21 | ETA:0:03:05 | Loss:0.745374375865573 | top1:50.074405670166016
31/269 Data:0.003 | Batch:0.582 | Total:0:00:30 | ETA:0:03:19 | Loss:0.7303792853509227 | top1:50.31501770019531
41/269 Data:2.306 | Batch:2.710 | Total:0:00:40 | ETA:0:03:48 | Loss:0.7215687007438846 | top1:50.6097526550293
51/269 Data:0.006 | Batch:0.558 | Total:0:00:49 | ETA:0:03:31 | Loss:0.7164715470052233 | top1:50.35232925415039
61/269 Data:0.553 | Batch:1.077 | Total:0:00:59 | ETA:0:03:16 | Loss:0.716273988856644 | top1:50.40983200073242
71/269 Data:0.003 | Batch:0.546 | Total:0:01:06 | ETA:0:02:24 | Loss:0.7136946001522978 | to

151/269 Data:0.005 | Batch:0.413 | Total:0:02:14 | ETA:0:01:36 | Loss:0.6949131469063411 | top1:50.434600830078125
161/269 Data:0.000 | Batch:0.336 | Total:0:02:20 | ETA:0:01:08 | Loss:0.6950240090767049 | top1:50.47554397583008
171/269 Data:0.001 | Batch:0.426 | Total:0:02:27 | ETA:0:01:10 | Loss:0.695209663165243 | top1:50.331233978271484
181/269 Data:0.001 | Batch:0.628 | Total:0:02:37 | ETA:0:01:22 | Loss:0.6952493286264535 | top1:50.29351043701172
191/269 Data:0.001 | Batch:0.640 | Total:0:02:43 | ETA:0:00:53 | Loss:0.6952157335755713 | top1:50.30472946166992
201/269 Data:0.004 | Batch:0.449 | Total:0:02:52 | ETA:0:01:01 | Loss:0.6951343749293047 | top1:50.347869873046875
211/269 Data:0.001 | Batch:0.622 | Total:0:03:00 | ETA:0:00:46 | Loss:0.6950282827372799 | top1:50.38877487182617
221/269 Data:0.001 | Batch:0.680 | Total:0:03:07 | ETA:0:00:34 | Loss:0.6949953307393449 | top1:50.39239501953125
231/269 Data:0.001 | Batch:0.621 | Total:0:03:13 | ETA:0:00:24 | Loss:0.69492601754861

31/269 Data:0.003 | Batch:0.668 | Total:0:00:31 | ETA:0:02:45 | Loss:0.6944546526478182 | top1:49.71017837524414
41/269 Data:0.012 | Batch:0.616 | Total:0:00:40 | ETA:0:03:34 | Loss:0.6945390163398371 | top1:49.73323059082031
51/269 Data:0.003 | Batch:0.663 | Total:0:00:50 | ETA:0:03:30 | Loss:0.6946637840831981 | top1:49.75490188598633
61/269 Data:0.678 | Batch:1.272 | Total:0:00:57 | ETA:0:02:39 | Loss:0.694601800598082 | top1:49.92955780029297
71/269 Data:0.003 | Batch:0.489 | Total:0:01:06 | ETA:0:03:03 | Loss:0.6944112450304166 | top1:50.2420768737793
81/269 Data:1.023 | Batch:1.647 | Total:0:01:17 | ETA:0:03:16 | Loss:0.6944169725900815 | top1:50.135032653808594
91/269 Data:0.001 | Batch:0.609 | Total:0:01:26 | ETA:0:03:04 | Loss:0.6945401777277936 | top1:50.060096740722656
101/269 Data:0.001 | Batch:0.689 | Total:0:01:35 | ETA:0:02:20 | Loss:0.6946061283054918 | top1:50.08895492553711
111/269 Data:0.014 | Batch:0.598 | Total:0:01:44 | ETA:0:02:26 | Loss:0.6946502612517761 | top1

...

161/269 Data:0.001 | Batch:0.608 | Total:0:02:14 | ETA:0:01:27 | Loss:0.025554681645116265 | top1:99.10472106933594
171/269 Data:0.000 | Batch:0.394 | Total:0:02:22 | ETA:0:01:23 | Loss:0.025667758775624442 | top1:99.11138916015625
181/269 Data:0.998 | Batch:1.401 | Total:0:02:30 | ETA:0:01:11 | Loss:0.025699658548325628 | top1:99.097900390625
191/269 Data:0.283 | Batch:0.967 | Total:0:02:37 | ETA:0:00:55 | Loss:0.025803634540923916 | top1:99.09603881835938
201/269 Data:0.001 | Batch:0.543 | Total:0:02:45 | ETA:0:00:58 | Loss:0.02574591160478506 | top1:99.0963134765625
211/269 Data:0.893 | Batch:1.552 | Total:0:02:53 | ETA:0:00:48 | Loss:0.02564543727961903 | top1:99.09471893310547
221/269 Data:0.005 | Batch:0.386 | Total:0:03:01 | ETA:0:00:44 | Loss:0.025496549900132337 | top1:99.10032653808594
231/269 Data:0.008 | Batch:0.677 | Total:0:03:09 | ETA:0:00:32 | Loss:0.02564084931433975 | top1:99.09700012207031
241/269 Data:1.782 | Batch:2.184 | Total:0:03:17 | ETA:0:00:24 | Loss:0.025625

31/269 Data:0.513 | Batch:0.861 | Total:0:00:21 | ETA:0:02:31 | Loss:0.021694881180601734 | top1:99.20614624023438
41/269 Data:0.003 | Batch:0.689 | Total:0:00:26 | ETA:0:02:14 | Loss:0.021226985777569252 | top1:99.23780059814453
51/269 Data:0.003 | Batch:0.651 | Total:0:00:33 | ETA:0:02:29 | Loss:0.022175109488706962 | top1:99.18045806884766
61/269 Data:0.014 | Batch:0.712 | Total:0:00:40 | ETA:0:02:22 | Loss:0.022848640476948904 | top1:99.1611099243164
71/269 Data:0.002 | Batch:0.680 | Total:0:00:46 | ETA:0:02:09 | Loss:0.02219811055651853 | top1:99.19123840332031
81/269 Data:0.004 | Batch:0.691 | Total:0:00:53 | ETA:0:02:15 | Loss:0.022670579211855377 | top1:99.17534637451172
91/269 Data:0.004 | Batch:0.719 | Total:0:01:01 | ETA:0:02:20 | Loss:0.022518192571434347 | top1:99.18012237548828
101/269 Data:0.003 | Batch:0.684 | Total:0:01:08 | ETA:0:01:50 | Loss:0.023688172331393354 | top1:99.13365936279297
111/269 Data:0.004 | Batch:0.668 | Total:0:01:15 | ETA:0:01:49 | Loss:0.023399125

181/269 Data:0.813 | Batch:1.343 | Total:0:02:27 | ETA:0:01:05 | Loss:0.02361762254931115 | top1:99.14105987548828
191/269 Data:0.696 | Batch:1.132 | Total:0:02:36 | ETA:0:01:08 | Loss:0.023110017462033797 | top1:99.16557312011719
201/269 Data:0.002 | Batch:0.679 | Total:0:02:43 | ETA:0:00:50 | Loss:0.023204411726343025 | top1:99.15655517578125
211/269 Data:0.001 | Batch:0.332 | Total:0:02:47 | ETA:0:00:26 | Loss:0.02363438818104989 | top1:99.13544464111328
221/269 Data:0.004 | Batch:0.684 | Total:0:02:55 | ETA:0:00:33 | Loss:0.023748930175978337 | top1:99.13568115234375
231/269 Data:0.003 | Batch:0.662 | Total:0:03:02 | ETA:0:00:28 | Loss:0.023756552805108102 | top1:99.12574768066406
241/269 Data:0.004 | Batch:0.679 | Total:0:03:08 | ETA:0:00:19 | Loss:0.023580795923958984 | top1:99.13771057128906
251/269 Data:0.001 | Batch:0.382 | Total:0:03:15 | ETA:0:00:13 | Loss:0.023608309011795073 | top1:99.13782501220703
261/269 Data:0.012 | Batch:0.657 | Total:0:03:22 | ETA:0:00:06 | Loss:0.02

51/269 Data:0.001 | Batch:0.370 | Total:0:00:35 | ETA:0:02:41 | Loss:0.02293960356573556 | top1:99.14981842041016
61/269 Data:0.002 | Batch:0.436 | Total:0:00:42 | ETA:0:02:27 | Loss:0.022662128772220163 | top1:99.1803207397461
71/269 Data:0.002 | Batch:0.329 | Total:0:00:48 | ETA:0:02:10 | Loss:0.022819572895712837 | top1:99.18573760986328
81/269 Data:0.002 | Batch:0.451 | Total:0:00:55 | ETA:0:02:15 | Loss:0.023774605779046262 | top1:99.13194274902344
91/269 Data:0.001 | Batch:0.611 | Total:0:01:02 | ETA:0:02:00 | Loss:0.02437076880852436 | top1:99.11573028564453
101/269 Data:0.001 | Batch:0.347 | Total:0:01:11 | ETA:0:02:39 | Loss:0.023763723496237014 | top1:99.12979125976562
111/269 Data:0.004 | Batch:0.487 | Total:0:01:24 | ETA:0:03:11 | Loss:0.02367587203814371 | top1:99.13780975341797
121/269 Data:0.004 | Batch:0.688 | Total:0:01:33 | ETA:0:02:14 | Loss:0.023674913864844473 | top1:99.1380386352539
131/269 Data:0.002 | Batch:0.372 | Total:0:01:39 | ETA:0:01:32 | Loss:0.0237439615

201/269 Data:0.934 | Batch:1.373 | Total:0:02:18 | ETA:0:00:49 | Loss:0.019807617227542238 | top1:99.267333984375
211/269 Data:1.056 | Batch:1.606 | Total:0:02:24 | ETA:0:00:37 | Loss:0.019985328959212838 | top1:99.25948333740234
221/269 Data:0.004 | Batch:0.519 | Total:0:02:30 | ETA:0:00:30 | Loss:0.02021341720633883 | top1:99.24349975585938
231/269 Data:0.001 | Batch:0.379 | Total:0:02:36 | ETA:0:00:24 | Loss:0.020570440559539122 | top1:99.24073028564453
241/269 Data:0.579 | Batch:0.984 | Total:0:02:44 | ETA:0:00:20 | Loss:0.020760818812571063 | top1:99.23983001708984
251/269 Data:0.001 | Batch:0.339 | Total:0:02:49 | ETA:0:00:11 | Loss:0.020594283087787577 | top1:99.24676513671875
261/269 Data:0.630 | Batch:1.032 | Total:0:02:55 | ETA:0:00:06 | Loss:0.02070071995954147 | top1:99.24419403076172
